<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluate RAG with LLM Evals</h1>

In this tutorial we will look into building a RAG pipeline and evaluating it with Phoenix Evals.

It has the the following sections:

1. Understanding Retrieval Augmented Generation (RAG).
2. Building RAG (with the help of a framework such as LlamaIndex).
3. Evaluating RAG with Phoenix Evals.

## Retrieval Augmented Generation (RAG)

LLMs are trained on vast datasets, but these will not include your specific data (things like company knowledge bases and documentation). Retrieval-Augmented Generation (RAG) addresses this by dynamically incorporating your data as context during the generation process. This is done not by altering the training data of the LLMs but by allowing the model to access and utilize your data in real-time to provide more tailored and contextually relevant responses.

In RAG, your data is loaded and prepared for queries. This process is called indexing. User queries act on this index, which filters your data down to the most relevant context. This context and your query then are sent to the LLM along with a prompt, and the LLM provides a response.

RAG is a critical component for building applications such a chatbots or agents and you will want to know RAG techniques on how to get data into your application.

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/RAG_Pipeline.png">

## Stages within RAG

There are five key stages within RAG, which will in turn be a part of any larger RAG application.

- **Loading**: This refers to getting your data from where it lives - whether it's text files, PDFs, another website, a database or an API - into your pipeline.
- **Indexing**: This means creating a data structure that allows for querying the data. For LLMs this nearly always means creating vector embeddings, numerical representations of the meaning of your data, as well as numerous other metadata strategies to make it easy to accurately find contextually relevant data.
- **Storing**: Once your data is indexed, you will want to store your index, along with any other metadata, to avoid the need to re-index it.

- **Querying**: For any given indexing strategy there are many ways you can utilize LLMs and data structures to query, including sub-queries, multi-step queries, and hybrid strategies. 
- **Evaluation**: A critical step in any pipeline is checking how effective it is relative to other strategies, or when you make changes. Evaluation provides objective measures on how accurate, faithful, and fast your responses to queries are.


## Build a RAG system 

Now that we have understood the stages of RAG, let's build a pipeline. We will use [LlamaIndex](https://www.llamaindex.ai/) for RAG and [Phoenix Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) for evaluation.


In [26]:
!pip install -qq "arize-phoenix[evals,llama-index]" "llama-index-llms-openai" "openai>=1" gcsfs nest_asyncio

For this tutorial we will be using OpenAI for creating synthetic data as well as for evaluation. 

In [1]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [2]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

import pandas as pd
import phoenix as px
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI

During this tutorial, we will capture all the data we need to evaluate our RAG pipeline using Phoenix Tracing. To enable this, simply start the phoenix application and instrument LlamaIndex.

In [3]:
px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Enable Phoenix tracing via `LlamaIndexInstrumentor`. Phoenix uses OpenInference traces - an open-source standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [4]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

### Load Data and Build an Index

Let's use an [essay by Paul Graham](https://www.paulgraham.com/worked.html) to build our RAG pipeline.

In [5]:
import tempfile
from urllib.request import urlretrieve

with tempfile.NamedTemporaryFile() as tf:
    urlretrieve(
        "https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/data/paul_graham/paul_graham_essay.txt",
        tf.name,
    )
    documents = SimpleDirectoryReader(input_files=[tf.name]).load_data()

In [6]:
# Define an LLM
llm = OpenAI(model="gpt-4o")

# Build index with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

Build a QueryEngine and start querying.

In [7]:
query_engine = vector_index.as_query_engine()

In [8]:
response_vector = query_engine.query("What did the author do growing up?")

Check the response that you get from the query.

In [9]:
response_vector.response

'The author focused on writing short stories and programming during their time before college.'

By default LlamaIndex retrieves two similar nodes/ chunks. You can modify that in `vector_index.as_query_engine(similarity_top_k=k)`.

Let's check the text in each of these retrieved nodes.

In [10]:
# First retrieved node
response_vector.source_nodes[0].get_text()

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then stack

In [11]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

"I remember taking the boys to the coast on a sunny day in 2015 and figuring out how to deal with some problem involving continuations while I watched them play in the tide pools. It felt like I was doing life right. I remember that because I was slightly dismayed at how novel it felt. The good news is that I had more moments like this over the next few years.\n\nIn the summer of 2016 we moved to England. We wanted our kids to see what it was like living in another country, and since I was a British citizen by birth, that seemed the obvious choice. We only meant to stay for a year, but we liked it so much that we still live there. So most of Bel was written in England.\n\nIn the fall of 2019, Bel was finally finished. Like McCarthy's original Lisp, it's a spec rather than an implementation, although like McCarthy's Lisp it's a spec expressed as code.\n\nNow that I could write essays again, I wrote a bunch about topics I'd had stacked up. I kept writing essays through 2020, but I also s

Remember that we are using Phoenix Tracing to capture all the data we need to evaluate our RAG pipeline. You can view the traces in the phoenix application.

In [12]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


We can access the traces by directly pulling the spans from the phoenix session.

In [13]:
spans_df = px.Client().get_spans_dataframe()

In [14]:
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

,name,span_kind,attributes.input.value,attributes.retrieval.documents
context.span_id,,,,
0fdb153cb1c346fb,BaseEmbedding.get_text_embedding_batch,EMBEDDING,None,None
29100b5af81c9e79,BaseEmbedding.get_query_embedding,EMBEDDING,None,None
8edc59832aaa8f6c,VectorIndexRetriever._retrieve,RETRIEVER,None,None
8dbd6ba584b59a21,BaseRetriever.retrieve,RETRIEVER,What did the author do growing up?,[{'document.content': 'What I Worked On Febru...
6b5a46b7ec759d9b,OpenAI.chat,LLM,None,None


Note that the traces have captured the documents that were retrieved by the query engine. This is nice because it means we can introspect the documents without having to keep track of them ourselves.

In [15]:
spans_with_docs_df = spans_df[spans_df["attributes.retrieval.documents"].notnull()]

In [16]:
spans_with_docs_df[["attributes.input.value", "attributes.retrieval.documents"]].head()

,attributes.input.value,attributes.retrieval.documents
context.span_id,,
8dbd6ba584b59a21,What did the author do growing up?,[{'document.content': 'What I Worked On Febru...


We have built a RAG pipeline and also have instrumented it using Phoenix Tracing. We now need to evaluate it's performance. We can assess our RAG system/query engine using Phoenix's LLM Evals. Let's examine how to leverage these tools to quantify the quality of our retrieval-augmented generation system.

## Evaluation

Evaluation should serve as the primary metric for assessing your RAG application. It determines whether the pipeline will produce accurate responses based on the data sources and range of queries.

While it's beneficial to examine individual queries and responses, this approach is impractical as the volume of edge-cases and failures increases. Instead, it's more effective to establish a suite of metrics and automated evaluations. These tools can provide insights into overall system performance and can identify specific areas that may require scrutiny.

In a RAG system, evaluation focuses on two critical aspects:

- **Retrieval Evaluation**: To assess the accuracy and relevance of the documents that were retrieved
- **Response Evaluation**: Measure the appropriateness of the response generated by the system when the context was provided.

### Generate Question Context Pairs

For the evaluation of a RAG system, it's essential to have queries that can fetch the correct context and subsequently generate an appropriate response.

For this tutorial, let's use Phoenix's `llm_generate` to help us create the question-context pairs.

First, let's create a dataframe of all the document chunks that we have indexed.

In [17]:
# Let's construct a dataframe of just the documents that are in our index
document_chunks_df = pd.DataFrame({"text": [node.get_text() for node in nodes]})
document_chunks_df = document_chunks_df.sample(10, random_state=42)
document_chunks_df.head()

,text
0,What I Worked On\n\nFebruary 2021\n\nBefore co...
5,"Only Harvard accepted me, so that was where I ..."
46,Surely the biggest source of stress in one's w...
31,"That's what I should have done, just gone off ..."
13,By subtly emphasizing such things you can make...


Now that we have the document chunks, let's prompt an LLM to generate us 3 questions per chunk. Note that you could manually solicit questions from your team or customers, but this is a quick and easy way to generate a large number of questions.

In [18]:
generate_questions_template = """\
Context information is below.

---------------------
{text}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
3 questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Output the questions in JSON format with the keys question_1, question_2, question_3.
"""

In [19]:
import json

from phoenix.evals import OpenAIModel, llm_generate


def output_parser(response: str, index: int):
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        return {"__error__": str(e)}


questions_df = llm_generate(
    dataframe=document_chunks_df,
    template=generate_questions_template,
    model=OpenAIModel(model="gpt-3.5-turbo"),
    output_parser=output_parser,
    concurrency=20,
)

llm_generate |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

In [20]:
questions_df.head()

,question_1,question_2,question_3
0,What were the two main things the author worke...,Describe the author's experience with programm...,How did the author's experience with programmi...
5,Explain the author's realization about AI duri...,Why did the author decide to focus on Lisp and...,Discuss the author's perspective on the relati...
46,What were the main sources of stress for the i...,How did the individual's role at YC differ fro...,Why did the individual continue to work hard a...
31,How did the narrator's decision to buy a house...,What changes did the narrator experience upon ...,What idea did the narrator have in the spring ...
13,How does the author describe the process of pa...,What was the author's experience at the Accade...,What unique feature did Interleaf add to their...


In [21]:
# Construct a dataframe of the questions and the document chunks
questions_with_document_chunk_df = pd.concat([questions_df, document_chunks_df], axis=1)
questions_with_document_chunk_df = questions_with_document_chunk_df.melt(
    id_vars=["text"], value_name="question"
).drop("variable", axis=1)
# If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
questions_with_document_chunk_df = questions_with_document_chunk_df[
    questions_with_document_chunk_df["question"].notnull()
]

The LLM has generated three questions per chunk. Let's take a quick look.

In [22]:
questions_with_document_chunk_df.head(10)

,text,question
0,What I Worked On\n\nFebruary 2021\n\nBefore co...,What were the two main things the author worke...
1,"Only Harvard accepted me, so that was where I ...",Explain the author's realization about AI duri...
2,Surely the biggest source of stress in one's w...,What were the main sources of stress for the i...
3,"That's what I should have done, just gone off ...",How did the narrator's decision to buy a house...
4,By subtly emphasizing such things you can make...,How does the author describe the process of pa...
5,I wrote an essay for myself to answer that que...,Explain the significance of skipping a step in...
6,This name didn't last long before it was repla...,Why did the author decide to name the new comp...
7,"I asked Jessica if she wanted to be president,...",Why did the founders of YC decide to recruit S...
8,She'd copy an obscure old painting out of a bo...,Explain the difference between painting still ...
9,and went on with the talk. But afterward it oc...,What was the realization that prompted the nar...


### Retrieval Evaluation

We are now prepared to perform our retrieval evaluations. We will execute the queries we generated in the previous step and verify whether or not that the correct context is retrieved.

In [24]:
# loop over the questions and generate the answers
for _, row in questions_with_document_chunk_df.iterrows():
    question = row["question"]
    response_vector = query_engine.query(question)
    print(f"Question: {question}\nAnswer: {response_vector.response}\n")

Question: What were the two main things the author worked on before college?
Answer: The author worked on writing and programming before college.

Question: Explain the author's realization about AI during their first year of grad school. What was the main issue they identified with the existing approach to AI?
Answer: The author realized during their first year of grad school that the existing approach to AI, which involved programs translating natural language into formal representations, was fundamentally flawed. They identified that these programs only demonstrated a subset of natural language that was a formal language, but not a comprehensive understanding of natural language. The main issue the author identified was the unbridgeable gap between what these programs could accomplish and true comprehension of natural language.

Question: What were the main sources of stress for the individual in their work at YC?
Answer: The main sources of stress for the individual in their work a

Now that we have executed the queries, we can start validating whether or not the RAG system was able to retrieve the correct context. Let's extract all the retrieved documents from the traces logged to phoenix. (For an in-depth explanation of how to export trace data from the phoenix runtime, consult the [docs](https://docs.arize.com/phoenix/how-to/extract-data-from-spans)).

In [25]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.Client())
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
8dbd6ba584b59a21 0                  d27185e18639588b7b5b9fe14b970c4f   
                 1                  d27185e18639588b7b5b9fe14b970c4f   
30ec1cee877102de 0                  b6143d327ed92ba59a4320c69842ae2c   
                 1                  b6143d327ed92ba59a4320c69842ae2c   
c44ecb537b7ec56c 0                  e5c16c8960da11224ab9f2c901edf3ef   
...                                                              ...   
44f56c03f34756a9 1                  2d925d9d088fe41e53dd870ceade8aaf   
6c57af9c3f94943e 0                  edc82829823a1b81e562061261ba14aa   
                 1                  edc82829823a1b81e562061261ba14aa   
aa9be09c29d6084f 0                  e9de73604ec075b8b9c013b5c1a94a44   
                 1                  e9de73604ec075b8b9c013b5c1a94a44   

                                                                                input  \
context.span_id  document_position                                                      
8dbd6ba584b59a21 0                                 What did the author do growing up?   
                 1                                 What did the author do growing up?   
30ec1cee877102de 0                  What were the two main things the author worke...   
                 1                  What were the two main things the author worke...   
c44ecb537b7ec56c 0                  Explain the author's realization about AI duri...   
...                                                                               ...   
44f56c03f34756a9 1                  How did the death of the founder's mother impa...   
6c57af9c3f94943e 0                  What is emphasized in painting still lives to ...   
                 1                  What is emphasized in painting still lives to ...   
aa9be09c29d6084f 0                  What was the unique approach taken by the narr...   
                 1                  What was the unique approach taken by the narr...   

                                                                            reference  \
context.span_id  document_position                                                      
8dbd6ba584b59a21 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  I remember taking the boys to the coast on a s...   
30ec1cee877102de 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Even then it took me several years to understa...   
c44ecb537b7ec56c 0                  Only Harvard accepted me, so that was where I ...   
...                                                                               ...   
44f56c03f34756a9 1                  "You know," he said, "you should make sure Y C...   
6c57af9c3f94943e 0                  She'd copy an obscure old painting out of a bo...   
                 1                  I remember that I answered the essay question ...   
aa9be09c29d6084f 0                  and went on with the talk. But afterward it oc...   
                 1                  The part we got first was to be an angel firm....   

                                    document_score  
context.span_id  document_position                  
8dbd6ba584b59a21 0                        0.816419  
                 1                        0.813929  
30ec1cee877102de 0                        0.841757  
                 1                        0.816314  
c44ecb537b7ec56c 0                        0.879293  
...                                            ...  
44f56c03f34756a9 1                        0.834362  
6c57af9c3f94943e 0                        0.875677  
                 1                        0.862963  
aa9be09c29d6084f 0                        0.900352  
                 1                        0.896497  

[62 rows x 4 columns]

Let's now use Phoenix's LLM Evals to evaluate the relevance of the retrieved documents with regards to the query. Note, we've turned on `explanations` which prompts the LLM to explain it's reasoning. This can be useful for debugging and for figuring out potential corrective actions.

In [26]:
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
)

relevance_evaluator = RelevanceEvaluator(OpenAIModel(model="gpt-4o"))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

run_evals |          | 0/62 (0.0%) | ⏳ 00:00<? | ?it/s

In [27]:
retrieved_documents_relevance_df.head()

label  score  \
context.span_id  document_position                     
8dbd6ba584b59a21 0                   relevant    1.0   
                 1                  unrelated    0.0   
30ec1cee877102de 0                   relevant    1.0   
                 1                  unrelated    0.0   
c44ecb537b7ec56c 0                   relevant    1.0   

                                                                          explanation  
context.span_id  document_position                                                     
8dbd6ba584b59a21 0                  To determine if the reference text is relevant...  
                 1                  To determine if the reference text is relevant...  
30ec1cee877102de 0                  To determine if the reference text is relevant...  
                 1                  To determine if the reference text is relevant...  
c44ecb537b7ec56c 0                  To determine if the reference text is relevant...

We can now combine the documents with the relevance evaluations to compute retrieval metrics. These metrics will help us understand how well the RAG system is performing.

In [28]:
documents_with_relevance_df = pd.concat(
    [retrieved_documents_df, retrieved_documents_relevance_df.add_prefix("eval_")], axis=1
)
documents_with_relevance_df

context.trace_id  \
context.span_id  document_position                                     
8dbd6ba584b59a21 0                  d27185e18639588b7b5b9fe14b970c4f   
                 1                  d27185e18639588b7b5b9fe14b970c4f   
30ec1cee877102de 0                  b6143d327ed92ba59a4320c69842ae2c   
                 1                  b6143d327ed92ba59a4320c69842ae2c   
c44ecb537b7ec56c 0                  e5c16c8960da11224ab9f2c901edf3ef   
...                                                              ...   
44f56c03f34756a9 1                  2d925d9d088fe41e53dd870ceade8aaf   
6c57af9c3f94943e 0                  edc82829823a1b81e562061261ba14aa   
                 1                  edc82829823a1b81e562061261ba14aa   
aa9be09c29d6084f 0                  e9de73604ec075b8b9c013b5c1a94a44   
                 1                  e9de73604ec075b8b9c013b5c1a94a44   

                                                                                input  \
context.span_id  document_position                                                      
8dbd6ba584b59a21 0                                 What did the author do growing up?   
                 1                                 What did the author do growing up?   
30ec1cee877102de 0                  What were the two main things the author worke...   
                 1                  What were the two main things the author worke...   
c44ecb537b7ec56c 0                  Explain the author's realization about AI duri...   
...                                                                               ...   
44f56c03f34756a9 1                  How did the death of the founder's mother impa...   
6c57af9c3f94943e 0                  What is emphasized in painting still lives to ...   
                 1                  What is emphasized in painting still lives to ...   
aa9be09c29d6084f 0                  What was the unique approach taken by the narr...   
                 1                  What was the unique approach taken by the narr...   

                                                                            reference  \
context.span_id  document_position                                                      
8dbd6ba584b59a21 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  I remember taking the boys to the coast on a s...   
30ec1cee877102de 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Even then it took me several years to understa...   
c44ecb537b7ec56c 0                  Only Harvard accepted me, so that was where I ...   
...                                                                               ...   
44f56c03f34756a9 1                  "You know," he said, "you should make sure Y C...   
6c57af9c3f94943e 0                  She'd copy an obscure old painting out of a bo...   
                 1                  I remember that I answered the essay question ...   
aa9be09c29d6084f 0                  and went on with the talk. But afterward it oc...   
                 1                  The part we got first was to be an angel firm....   

                                    document_score    eval_label  eval_score  \
context.span_id  document_position                                             
8dbd6ba584b59a21 0                        0.816419      relevant         1.0   
                 1                        0.813929     unrelated         0.0   
30ec1cee877102de 0                        0.841757      relevant         1.0   
                 1                        0.816314     unrelated         0.0   
c44ecb537b7ec56c 0                        0.879293      relevant         1.0   
...                                            ...           ...         ...   
44f56c03f34756a9 1                        0.834362      relevant         1.0   
6c57af9c3f94943e 0                        0.875677      relevant         1.0   

Let's compute Normalized Discounted Cumulative Gain [NCDG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) at 2 for all our retrieval steps.  In information retrieval, this metric is often used to measure effectiveness of search engine algorithms and related applications.

In [29]:
import numpy as np
from sklearn.metrics import ndcg_score


def _compute_ndcg(df: pd.DataFrame, k: int):
    """Compute NDCG@k in the presence of missing values"""
    n = max(2, len(df))
    eval_scores = np.zeros(n)
    doc_scores = np.zeros(n)
    eval_scores[: len(df)] = df.eval_score
    doc_scores[: len(df)] = df.document_score
    try:
        return ndcg_score([eval_scores], [doc_scores], k=k)
    except ValueError:
        return np.nan


ndcg_at_2 = pd.DataFrame(
    {"score": documents_with_relevance_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
)

In [30]:
ndcg_at_2

,score
context.span_id,
06f24ea08188db3f,1.00000
0dfd91f61d5bb9fa,0.00000
0ecc535827f5e5b4,1.00000
1d327c6b2b9f24b1,1.00000
28224dd075e66227,1.00000
303070483efb4d33,1.00000
30ec1cee877102de,1.00000
3753ff1b2ec9415e,1.00000
388ca6253bd842a4,0.63093


Let's also compute precision at 2 for all our retrieval steps.

In [31]:
precision_at_2 = pd.DataFrame(
    {
        "score": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) / 2
        )
    }
)

In [32]:
precision_at_2

,score
context.span_id,
06f24ea08188db3f,1.0
0dfd91f61d5bb9fa,0.0
0ecc535827f5e5b4,1.0
1d327c6b2b9f24b1,1.0
28224dd075e66227,0.5
303070483efb4d33,0.5
30ec1cee877102de,0.5
3753ff1b2ec9415e,1.0
388ca6253bd842a4,0.5


Lastly, let's compute whether or not a correct document was retrieved at all for each query (e.g. a hit)

In [33]:
hit = pd.DataFrame(
    {
        "hit": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) > 0
        )
    }
)

Let's now view the results in a combined dataframe.

In [47]:
retrievals_df = px.Client().get_spans_dataframe(
    "span_kind == 'RETRIEVER' and input.value is not None"
)
rag_evaluation_dataframe = pd.concat(
    [
        retrievals_df["attributes.input.value"],
        ndcg_at_2.add_prefix("ncdg@2_"),
        precision_at_2.add_prefix("precision@2_"),
        hit,
    ],
    axis=1,
)
rag_evaluation_dataframe

,attributes.input.value,ncdg@2_score,precision@2_score,hit
context.span_id,,,,
8dbd6ba584b59a21,What did the author do growing up?,1.00000,0.5,True
30ec1cee877102de,What were the two main things the author worke...,1.00000,0.5,True
c44ecb537b7ec56c,Explain the author's realization about AI duri...,1.00000,1.0,True
0ecc535827f5e5b4,What were the main sources of stress for the i...,1.00000,1.0,True
67a87949fe836aee,How did the narrator's decision to buy a house...,1.00000,1.0,True
3753ff1b2ec9415e,How does the author describe the process of pa...,1.00000,1.0,True
b328eb721e4d4628,Explain the significance of skipping a step in...,1.00000,0.5,True
388ca6253bd842a4,Why did the author decide to name the new comp...,0.63093,0.5,True
4db4ae606a455c71,Why did the founders of YC decide to recruit S...,1.00000,1.0,True


### Observations

Let's now take our results and aggregate them to get a sense of how well our RAG system is performing.

In [35]:
# Aggregate the scores across the retrievals
results = rag_evaluation_dataframe.mean(numeric_only=True)
results

ncdg@2_score         0.847157
precision@2_score    0.677419
dtype: float64

As we can see from the above numbers, our RAG system is not perfect, there are times when it fails to retrieve the correct context within the first two documents. At other times the correct context is included in the top 2 results but non-relevant information is also included in the context. This is an indication that we need to improve our retrieval strategy. One possible solution could be to increase the number of documents retrieved and then use a more sophisticated ranking strategy (such as a reranker) to select the correct context.

We have now evaluated our RAG system's retrieval performance. Let's send these evaluations to Phoenix for visualization. By sending the evaluations to Phoenix, you will be able to view the evaluations alongside the traces that were captured earlier.

In [36]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=ndcg_at_2, eval_name="ndcg@2"),
    SpanEvaluations(dataframe=precision_at_2, eval_name="precision@2"),
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="relevance"),
)

### Response Evaluation

The retrieval evaluations demonstrates that our RAG system is not perfect. However, it's possible that the LLM is able to generate the correct response even when the context is incorrect. Let's evaluate the responses generated by the LLM.

In [37]:
from phoenix.session.evaluation import get_qa_with_reference

qa_with_reference_df = get_qa_with_reference(px.Client())
qa_with_reference_df

,input,output,reference
context.span_id,,,
8849e05d729e48c5,What did the author do growing up?,The author focused on writing short stories an...,What I Worked On\n\nFebruary 2021\n\nBefore co...
d8ded17b60ecfdda,What were the two main things the author worke...,The author worked on writing and programming b...,What I Worked On\n\nFebruary 2021\n\nBefore co...
2d14691ecacf94df,Explain the author's realization about AI duri...,The author realized during their first year of...,"Only Harvard accepted me, so that was where I ..."
2101659b600a75e0,What were the main sources of stress for the i...,The main sources of stress for the individual ...,Surely the biggest source of stress in one's w...
589fa2425d508997,How did the narrator's decision to buy a house...,The narrator's decision to buy a house in the ...,"That's what I should have done, just gone off ..."
dc5874ca6a32dbad,How does the author describe the process of pa...,The author describes painting still lives as a...,She'd copy an obscure old painting out of a bo...
22394abaaa4a803e,Explain the significance of skipping a step in...,Skipping the step of using punched cards as in...,I was puzzled by the 1401. I couldn't figure o...
4a92541f536e1d3e,Why did the author decide to name the new comp...,The author decided to name the new company 'As...,This name didn't last long before it was repla...
ba4e096a43329c68,Why did the founders of YC decide to recruit S...,The founders of YC decided to recruit Sam Altm...,"I asked Jessica if she wanted to be president,..."


Now that we have a dataset of the question, context, and response (input, reference, and output), we now can measure how well the LLM is responding to the queries. For details on the QA correctness evaluation, see the [LLM Evals documentation](https://docs.arize.com/phoenix/llm-evals/running-pre-tested-evals/q-and-a-on-retrieved-data).

In [38]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    run_evals,
)

qa_evaluator = QAEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))
hallucination_evaluator = HallucinationEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))

qa_correctness_eval_df, hallucination_eval_df = run_evals(
    evaluators=[qa_evaluator, hallucination_evaluator],
    dataframe=qa_with_reference_df,
    provide_explanation=True,
    concurrency=20,
)

run_evals |          | 0/62 (0.0%) | ⏳ 00:00<? | ?it/s

In [39]:
qa_correctness_eval_df.head()

,label,score,explanation
context.span_id,,,
8849e05d729e48c5,correct,1,The reference text clearly states that before ...
d8ded17b60ecfdda,correct,1,The reference text clearly states that before ...
2d14691ecacf94df,correct,1,The process to determine if the answer is corr...
2101659b600a75e0,incorrect,0,The reference text clearly identifies Hacker N...
589fa2425d508997,correct,1,"To determine if the answer is correct, we anal..."


In [40]:
hallucination_eval_df.head()

,label,score,explanation
context.span_id,,,
8849e05d729e48c5,factual,0,The reference text clearly states that before ...
d8ded17b60ecfdda,factual,0,The reference text explicitly states that befo...
2d14691ecacf94df,factual,0,The answer accurately reflects the content and...
2101659b600a75e0,hallucinated,1,The reference text clearly identifies Hacker N...
589fa2425d508997,factual,0,The answer is factual based on the reference t...


#### Observations

Let's now take our results and aggregate them to get a sense of how well the LLM is answering the questions given the context.

In [41]:
qa_correctness_eval_df.mean(numeric_only=True)

score    0.870968
dtype: float64

In [42]:
hallucination_eval_df.mean(numeric_only=True)

score    0.064516
dtype: float64

Our QA Correctness score of `0.91` and a Hallucinations score `0.05` signifies that the generated answers are correct ~91% of the time and that the responses contain hallucinations 5% of the time - there is room for improvement. This could be due to the retrieval strategy or the LLM itself. We will need to investigate further to determine the root cause.

Since we have evaluated our RAG system's QA performance and Hallucinations performance, let's send these evaluations to Phoenix for visualization.

In [43]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=qa_correctness_eval_df, eval_name="Q&A Correctness"),
    SpanEvaluations(dataframe=hallucination_eval_df, eval_name="Hallucination"),
)

We now have sent all our evaluations to Phoenix. Let's go to the Phoenix application and view the results! Since we've sent all the evals to Phoenix, we can analyze the results together to make a determination on whether or not poor retrieval or irrelevant context has an effect on the LLM's ability to generate the correct response.

In [44]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


## Conclusion

We have explored how to build and evaluate a RAG pipeline using LlamaIndex and Phoenix, with a specific focus on evaluating the retrieval system and generated responses within the pipelines. 

Phoenix offers a variety of other evaluations that can be used to assess the performance of your LLM Application. For more details, see the [LLM Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) documentation.